In [1]:
import numpy as np
import pandas as pd
import RiskSLIM as risk
import csv

from riskslim.helper_functions import load_data_from_csv, print_model
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.utils import shuffle

### Lasso Feature Selection

In [2]:
## load stumps data
data = pd.read_csv("~/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/data/broward_stumps.csv")
X, Y = data.loc[:,:'five_year>=1'], data['recid_property6'].values
cols = X.columns

In [10]:
## lasso
lasso = Lasso(random_state=816, alpha=0.01).fit(X, Y)
selected_features = cols[lasso.coef_ != 0].tolist()
len(selected_features), roc_auc_score(Y, lasso.predict(X))

(10, 0.7532382565095003)

#### subset features

In [11]:
### Subset features
selected_features.insert(0, 'recid_property6')
sub_data = data[selected_features]
sub_X, sub_Y = sub_data.iloc[:,1:], sub_data.iloc[:,0].values
sub_X.insert(0, '(Intercept)', 1)

### Cross Validation

In [12]:
SLIM_X, SLIM_Y = sub_X.values, sub_Y.reshape(-1,1)
variable_names = sub_X.columns.tolist()
outcome_name = 'recid_property6'
sample_weights = np.repeat(1, len(sub_Y))

In [13]:
#cv = KFold(n_splits=5, random_state=816, shuffle=True)
cv = KFold(n_splits=5, random_state=816, shuffle=True)
train_auc, test_auc = [], []

i = 0
for train, test in cv.split(SLIM_X, SLIM_Y):
    
    ## subset train data & store test data
    X_train, Y_train = SLIM_X[train], SLIM_Y[train]
    X_test, Y_test = SLIM_X[test], SLIM_Y[test]
    sample_weights_train, sample_weights_test = sample_weights[train], sample_weights[test]

    ## create new data dictionary
    new_train_data = {
        'X': X_train,
        'Y': Y_train,
        'variable_names': variable_names,
        'outcome_name': outcome_name,
        'sample_weights': sample_weights_train
    }
        
    ## fit the model
    model_info, mip_info, lcpa_info = risk.risk_slim(new_train_data, max_coefficient=20, max_L0_value=10, 
                                                c0_value=1e-2, max_runtime=200)
    print_model(model_info['solution'], new_train_data)
    
    ## change data format
    X_train, X_test = X_train[:,1:], X_test[:,1:] ## remove the first column, which is "intercept"
    Y_train[Y_train == -1] = 0 ## change -1 to 0
    Y_test[Y_test == -1] = 0
    
    ## probability & accuracy
    train_prob = risk.riskslim_prediction(X_train, cols, model_info).reshape(-1,1)
    test_prob = risk.riskslim_prediction(X_test, cols, model_info).reshape(-1,1)
    
    ## AUC
    train_auc.append(roc_auc_score(Y_train, train_prob))
    test_auc.append(roc_auc_score(Y_test, test_prob))

setting c0 = 0.0 to ensure that intercept is not penalized
08/17/19 @ 05:49 PM | 501 rows in lookup table
08/17/19 @ 05:49 PM | ------------------------------------------------------------
08/17/19 @ 05:49 PM | runnning initialization procedure
08/17/19 @ 05:49 PM | ------------------------------------------------------------
08/17/19 @ 05:49 PM | CPA produced 2 cuts
08/17/19 @ 05:49 PM | running naive rounding on 74 solutions
08/17/19 @ 05:49 PM | best objective value: 0.1782
08/17/19 @ 05:49 PM | rounding produced 1 integer solutions
08/17/19 @ 05:49 PM | best objective value is 0.2054
08/17/19 @ 05:49 PM | running sequential rounding on 74 solutions
08/17/19 @ 05:49 PM | best objective value: 0.1782
08/17/19 @ 05:49 PM | sequential rounding produced 2 integer solutions
08/17/19 @ 05:49 PM | best objective value: 0.1988
08/17/19 @ 05:49 PM | polishing 3 solutions
08/17/19 @ 05:49 PM | best objective value: 0.1988
08/17/19 @ 05:49 PM | polishing produced 3 integer solutions
08/17/19 @

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
08/17/19 @ 05:49 PM | adding 73 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.1942.
Tried aggregator 2 times.
MIP Presolve eliminated 1 rows and 1 columns.
Aggregator did 1 substitutions.
Reduced MIP has 10 rows, 22 columns, and 20 nonzeros.
Reduced MIP has 10 binaries, 11 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (0.02 ticks)
Probing time = 0.00 sec. (0.00 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.01 ticks)

        Nodes                                         Cuts/
   Node  Left   

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
08/17/19 @ 05:49 PM | adding 79 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.2118.
Tried aggregator 2 times.
MIP Presolve eliminated 1 rows and 1 columns.
Aggregator did 1 substitutions.
Reduced MIP has 10 rows, 22 columns, and 20 nonzeros.
Reduced MIP has 10 binaries, 11 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.02 ticks)
Probing time = 0.00 sec. (0.00 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.01 ticks)

        Nodes                                         Cuts/
   Node  Left   

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
08/17/19 @ 05:49 PM | adding 67 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.2053.
Tried aggregator 2 times.
MIP Presolve eliminated 1 rows and 1 columns.
Aggregator did 1 substitutions.
Reduced MIP has 10 rows, 22 columns, and 20 nonzeros.
Reduced MIP has 10 binaries, 11 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.02 ticks)
Probing time = 0.00 sec. (0.00 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.01 ticks)

        Nodes                                         Cuts/
   Node  Left   

ValueError: max() arg is an empty sequence

In [14]:
np.mean(train_auc), np.mean(test_auc)

(0.6567830030448659, 0.6537021125893077)

### Save Results

In [15]:
#log model results to the model performance folder, as per standards
path = "C:\\Users\\binha\\Documents\\Duke\\Cynthia Research\\KY-analysis-mytrials\\broward\\broward models\\model results\\Advanced Models\\Six Month\\"
results = [["Property", np.mean(train_auc), np.std(train_auc), np.mean(test_auc), np.std(test_auc)]]
with open(path + 'Six Month RiskSLIM.csv', 'a') as writeFile:
    writer = csv.writer(writeFile)
    writer.writerows(results)